In [2]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
# from lib2to3.pgen2 import driver
from IPython.display import Javascript
import pandas as pd
import numpy as np
import csv
import os
import itertools as it

---

<center><h1><b>MarkUp</b></h1></center>

---

- After creating the "Udemy_URLS" csv file using crawling, we extracted the diffrent urls from the file and using selenium and Beautiful soup(we needed the selelnium because of the problem of "dynamic Javascript") created a loop which in every iteration we pulled URL, appended https and started the driver with this url and got the html with all the elements.
- After we got the html content we started extracting the elements information like: best_seller, course_rating_avarage, course_price, and etc...
- If the website failed to rise then we move forward to the next url
- If any of the elemets failed during extaraction we saved -1 instead
- In the end, we saved all the inforamtion to the CSV file called 'UDEMY_DATA'

In [ ]:
global urls
urls = pd.read_csv('./Udemy_URLS.csv')
global url
url = ''
global start_index
start_index = 26008
global end_index
end_index = 26009

def forward_url():
    global url
    global start_index
    global end_index
    
    start_index = start_index + 1
    end_index = end_index + 1
    with open('./Udemy_URLS.csv') as f:
        reader=csv.reader(f)
        url = 'https://' + next(it.islice(reader, start_index, end_index))[0]
    run_web_driver()
    
def run_web_driver():
    global url
    try:
        opts = Options()
        os.environ['WDM_LOG_LEVEL'] = '0'
        opts.add_argument("start-maximized")
        opts.add_experimental_option('excludeSwitches', ['enable-logging'])
        opts.add_argument("start-maximized")
        opts.add_experimental_option("excludeSwitches", ["enable-automation"])
        opts.add_experimental_option('excludeSwitches', ['enable-logging'])
        opts.add_experimental_option('useAutomationExtension', False)
        opts.add_argument('--disable-blink-features=AutomationControlled')
#         opts.add_argument('--headless')
        opts.add_argument('--disable-gpu')  # Last I checked this was necessary.
#         opts.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36")
#         opts.add_argument("user-agent=" + agent)
        driver = webdriver.Chrome(chrome_options = opts)
        driver.maximize_window()
        time.sleep(8)
        driver.get(url)
        time.sleep(10)
        
        soup = BeautifulSoup(driver.page_source, 'lxml')
        driver.close()
        
        #best_seller
        best_seller = -1
        try:  
            best_seller_text = ''
            for a in soup.select('div.clp-lead__badge-ratings-enrollment'):
                for b in soup.select('div.clp-lead__element-item'):
                    best_seller_text = b.text
                    break
                
                if(best_seller_text == 'Bestseller'):
                    best_seller = 1
                else:
                    best_seller = 0
                    break
#             print(best_seller)
        except Exception as e: 
            best_seller = -1
#             print("best seller fail: ")#+ str(e))
            
        
        #course_rating_avarage
        course_rating_avarage = -1
        try:
            course_rating_avarage_elem = soup.find('span', {"class" : 'udlite-heading-sm star-rating--rating-number--2o8YM'})
            course_rating_avarage = course_rating_avarage_elem.string
#             print(course_rating_avarage)
        except Exception as e: 
            course_rating_avarage = -1
#             print("course_rating_avergae fail: ")#+ str(e))
        
        #course_price
        course_price = -1
        try:
            course_price_elem = soup.find("div",{"data-purpose":"course-price-text"})
            course_price_text = course_price_elem.text
            course_price = course_price_text.replace("Current price₪","")
#             print(course_price)
        except Exception as e: 
            course_price = -1
#             print("course_price fail: ")#+ str(e))
  
        #num_of_buyers_students
        num_of_buyers_students = -1
        try:
            num_of_buyers_students_elem = soup.find('div', {"class" : 'enrollment'})
            num_of_buyers_students_text = num_of_buyers_students_elem.string
            num_of_buyers_students = num_of_buyers_students_text.replace(" students", "")
#             print(num_of_buyers_students)
        except Exception as e: 
            num_of_buyers_students = -1
#             print("num_of_buyers_studens fail: ")#+ str(e))

        #amount_of_instructor_studnets
        amount_of_instructor_studnets = -1
        try:
            counter = 0
            for d in soup.select('div.instructor--instructor__image-and-stats--1IqE7'):
                for i in d.select('div.udlite-block-list-item-content'):  
                    counter = counter + 1
                    if(counter == 3):
                        amount_of_instructor_studnets = i.text.replace(" Students", "")
                        counter = 0
#             print(amount_of_instructor_studnets)
        except Exception as e: 
            amount_of_instructor_studnets = -1
#             print("amount_of_instructor_students fail: ")#+ str(e))
        
        #num_articles
        num_articles = -1
        try:
            num_articles_elem = soup.find("span",{"data-purpose" : "num-articles"})
            num_articles_text = num_articles_elem.text
            num_articles = num_articles_text.replace(" articles","").replace(" article", "")
#             print(num_articles)
        except Exception as e: 
            num_articles = -1
#             print("num_articles fail: ")#+ str(e))
        
        #video_time_length
        video_time_length = -1
        try:
            video_time_length_elem = soup.find("span", {"data-purpose":"video-content-length"})
            video_time_length_text = video_time_length_elem.text
            video_time_length = video_time_length_text.replace(" hours on-demand video","")
#             print(video_time_length)
        except Exception as e: 
            video_time_length = -1
#             print("video_time_length fail: ")#+ str(e))
        
        #number_of_languages
        number_of_languages = -1
        try:
            number_of_languages_elem = soup.find('button', attrs={'class':'udlite-btn udlite-btn-medium udlite-btn-link udlite-text-sm caption--more-button--3_cNb'})
            if(number_of_languages_elem == None):
                number_of_languages = 2
            else:
                number_of_languages = int(number_of_languages_elem.text.replace(" more", "")) + 2
#             print(number_of_languages)
        except Exception as e: 
            number_of_languages = -1
#             print("number_of_languages fail: ")#+ str(e))
    
        #last_update
        last_update = -1
        try:
            last_update_elem = soup.find('div', attrs={'class':'last-update-date'})
            last_update = last_update_elem.text.replace("Last updated ","")
#             print(last_update)
        except Exception as e: 
            last_update = -1
#             print("last_update fail: ")#+ str(e))
        
        #instructor_rank
        instructor_rank = -1
        try:
            for d in soup.select('div.instructor--instructor__image-and-stats--1IqE7'):
                for i in d.select('div.udlite-block-list-item-content'):        
                    instructor_rank = i.text.replace(" Instructor Rating","")
                    break
#             print(instructor_rank)
        except Exception as e: 
            instructor_rank = -1
#             print("instructor_rank fail: ")#+ str(e))
        
        #amount_of_what_you_will_learn_count
        amount_of_what_you_will_learn_count = -1
        try:
            amount_of_what_you_will_learn_ul = soup.find('ul', attrs={'class': 'unstyled-list udlite-block-list what-you-will-learn--objectives-list--2cWZN what-you-will-learn--objectives-list-two-column-layout--7vG__'})
            amount_of_what_you_will_learn_count = 0
            for li in amount_of_what_you_will_learn_ul.findAll('li'):
                amount_of_what_you_will_learn_count = amount_of_what_you_will_learn_count + 1
#             print(amount_of_what_you_will_learn_count)
        except Exception as e: 
            amount_of_what_you_will_learn_count = -1
#             print("amount_of_what_you_will_learn_count fail: ")#+ str(e))
        
        #amount_of_requirments_count
        amount_of_requirments_count = -1
        try:
            amount_of_requirments_ul = soup.find('ul', attrs={'class': 'unstyled-list udlite-block-list'})
            amount_of_requirments_count = 0
            for li in amount_of_requirments_ul.findAll('li'):
                amount_of_requirments_count = amount_of_requirments_count + 1
#             print(amount_of_requirments_count)
        except Exception as e: 
            amount_of_requirments_count = -1
#             print("amount_of_requirments fail: ")#+ str(e))
        
        #amount_of_companies_support
        amount_of_companies_support = -1
        try:
            amount_of_companies_support = 0
            for d in soup.select('div.curated-for-ufb-notice--top-companies-logos--2_LsV'):
                for i in d.select('img'):
                    amount_of_companies_support = amount_of_companies_support + 1
#             print(amount_of_companies_support)
        except Exception as e: 
            amount_of_companies_support = -1
#             print("amount_of_companies_support fail: ")#+ str(e))
        
        #course_rating_amount
        course_rating_amount = -1
        try:
            course_rating_amount_elem =  soup.find('a',{"class":'styles--rating-wrapper--5a0Tr'})
            children = course_rating_amount_elem.findChildren("span" , recursive=False)
            course_rating_amount = children[-1].text.replace(" ratings)","")
            course_rating_amount = course_rating_amount.replace("(","")
#             print(course_rating_amount)
        except Exception as e: 
            course_rating_amount = -1
#             print("course_rating_amount fail: ")#+ str(e))

        
        data = [[best_seller, course_rating_avarage, course_price, num_of_buyers_students, amount_of_instructor_studnets, num_articles, video_time_length, number_of_languages, last_update, instructor_rank, amount_of_what_you_will_learn_count, amount_of_requirments_count, amount_of_companies_support, course_rating_amount]]
        columns = ["best_seller", "course_rating_avarage", "course_price", "num_of_buyers_students", "amount_of_instructor_studnets", "num_articles", "video_time_length", "number_of_languages", "last_update", "instructor_rank", "amount_of_what_you_will_learn_count", "amount_of_requirments_count", "amount_of_companies_support", "course_rating_amount"]
        df = pd.DataFrame(data, columns = columns)
        
        file_name = 'UDEMY_DATA.csv'
        if not os.path.isfile(file_name):
            df.to_csv(file_name, header=columns ,index=False)
        else:
            df.to_csv(file_name, mode='a', header=False, index=False)
        
        forward_url()
    except Exception as e: 
        print("Failed in run_web_driver: ")#+ str(e))
        forward_url()

forward_url()
run_web_driver()